# Differential Evolution

In [939]:
import numpy as np
from numpy.random import choice, uniform

In [940]:
def init_population(pop_size, var_count, min_bound, max_bound):
    return uniform(low=min_bound, high=max_bound, size=(pop_size, var_count))

In [941]:
def calc_fitness_scores(population):
    scores = []

    for vector in population:
        # Rosenbrock's Function
        score = 100 * (vector[1] - vector[0]**2)**2 + (vector[0] - 1)**2
        scores.append(score)

    return scores

In [942]:
def differential_mutation(idx, population, min_bound, max_bound, scaling_factor = 0.5):
    used_idx = [idx]
    used_idx_no = 3

    while len(used_idx) <= used_idx_no:
        idx_selected = False
        while idx_selected == False:
            vector_idx = choice(np.array(range(len(population))), 1, replace=False)[0]
            if vector_idx not in used_idx:
                idx_selected = True
                used_idx.append(vector_idx)

    target_vector = used_idx[1]
    ran_select_1 = used_idx[2]
    ran_select_2 = used_idx[3]

    trial_vector = []
    trial_vec_loc_1 = population[target_vector][0] + scaling_factor * (population[ran_select_1][0] - population[ran_select_2][0])
    if trial_vec_loc_1 < min_bound:
        trial_vector.append(min_bound)
    elif trial_vec_loc_1 > max_bound:
        trial_vector.append(max_bound)
    else:
        trial_vector.append(trial_vec_loc_1)
    trial_vec_loc_2 = population[target_vector][1] + scaling_factor * (population[ran_select_1][1] - population[ran_select_2][1])
    if trial_vec_loc_2 < min_bound:
        trial_vector.append(min_bound)
    elif trial_vec_loc_2 > max_bound:
        trial_vector.append(max_bound)
    else:
        trial_vector.append(trial_vec_loc_2)

    return trial_vector

In [943]:
def crossover(idx, population, trial_vector, crossover_prob = 0.7):
    parent_vector = population[idx]
    offspring_vector = []
    var_len = len(parent_vector)
    trial_vec_select_idx = list(choice(np.array(range(var_len)), 1, replace=False))
    for i in range(var_len):
        if uniform(0, 1) < crossover_prob and i not in trial_vec_select_idx:
            trial_vec_select_idx.append(i)
    for j in range(var_len):
        if j in trial_vec_select_idx:
            offspring_vector.append(trial_vector[j])
        else:
            offspring_vector.append(parent_vector[j])

    return offspring_vector

In [944]:
def selection(idx, offspring_vector, population, scores):
    sub_pop = [population[idx], offspring_vector]
    sub_scores = calc_fitness_scores(sub_pop)
    if sub_scores[1] < sub_scores[0]:
        population[idx] = offspring_vector
        scores[idx] = sub_scores[1]

    return

In [945]:
def differential_evolution(var_count, min_bound, max_bound, pop_size, generations, scaling_factor = 0.5, crossover_prob = 0.7):
    population = init_population(pop_size, var_count, min_bound, max_bound)
    scores = calc_fitness_scores(population)
    for i in range(generations):
        print(f"******************************* Generation {i + 1} *******************************")
        print("Population-------------------------------------------------------Scores")
        for idx, vector in enumerate(population):
            trial_vector = differential_mutation(idx, population, min_bound, max_bound)
            offspring_vector = crossover(idx, population, trial_vector, 0.7)
            selection(idx, offspring_vector, population, scores)
            print(str(population[idx])+"                              "+str(scores[idx]))
        best_idx = scores.index(min(scores))
        print("===============================================================================")
        print(f"Best Vector: {population[best_idx]}----------------Score: {scores[best_idx]}")
        print("===============================================================================")

In [946]:
differential_evolution(2, -30, 30, 20, 100)

******************************* Generation 1 *******************************
Population-------------------------------------------------------Scores
[16.21313992 -3.04012979]                              7070833.43550172
[-9.8293077  24.79182147]                              515978.33450100425
[ 8.57156989 -9.10271454]                              681912.5462414948
[-19.31612953  20.12156515]                              12460698.183097191
[-1.60669032 -8.7325734 ]                              12807.51597093914
[-11.63606939  10.88138322]                              1550601.2139216687
[-6.64943229 30.        ]                              20264.99350090161
[14.88648507 22.86976235]                              3949859.199139846
[-4.73576979 25.05259443]                              722.0029757805598
[15.40238252 27.90624904]                              4381992.510568969
[20.16865742 18.59813767]                              15068475.051002175
[-19.39630741 -29.17967486]              